In [21]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from SALib.analyze import sobol
import warnings
from ema_workbench.analysis import feature_scoring
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from optimization_tuned import epsilon_nondominated,HypervolumeMetric, Hypervolume
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
    perform_experiments,
    Samplers,
    SequentialEvaluator,
)
from dike_model_function import DikeNetwork
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time
from ema_workbench.analysis import prim
from ema_workbench.em_framework.parameters import Constant
import pickle
warnings.filterwarnings("ignore")

In [22]:
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [25]:
for outcome in dike_model.outcomes:
    print(outcome)

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage', 'A.1_Dike Investment Costs'), function=<function sum_over at 0x0000020AAEBB5080>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths',), function=<function sum_over at 0x0000020AAEBB5080>)
ScalarOutcome('A.1_Expected Annual Damage', variable_name=('A.1_Expected Annual Damage',), function=<function sum_over at 0x0000020AAEBB5080>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage', 'A.2_Dike Investment Costs'), function=<function sum_over at 0x0000020AAEBB5080>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths',), function=<function sum_over at 0x0000020AAEBB5080>)
ScalarOutcome('A.2_Expected Annual Damage', variable_name=('A.2_Expected Annual Damage',), function=<function sum_over at 0x0000020AAEBB5080>)
ScalarOutcome('A.3 Total Costs', variable_name=('A.3_Expected Annual Damage', 'A.3_Dike Investme

In [26]:
dike_model2, planning_steps2 = get_model_for_problem_formulation(6)

In [27]:
for outcome in dike_model2.outcomes:
    print(outcome)

ScalarOutcome('A.5_Expected Number of Deaths', variable_name=('A.5_Expected Number of Deaths',), function=<function sum_over at 0x0000020AAEBB5080>)
ScalarOutcome('A.5_Expected Annual Damage', variable_name=('A.5_Expected Annual Damage',), function=<function sum_over at 0x0000020AAEBB5080>)
ScalarOutcome('Expected Evacuation Costs', variable_name=('Expected Evacuation Costs',), function=<function sum_over at 0x0000020AAEBB5080>)
ScalarOutcome('Expected Annual Damage', variable_name=('A.1_Expected Annual Damage', 'A.2_Expected Annual Damage', 'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage', 'A.5_Expected Annual Damage'), function=<function sum_over at 0x0000020AAEBB5080>)
ScalarOutcome('Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths', 'A.3_Expected Number of Deaths', 'A.4_Expected Number of Deaths', 'A.5_Expected Number of Deaths'), function=<function sum_over at 0x0000020AAEBB5080>)


## Scenarios to determine epsilon spacing

In [3]:
ScenResul = pd.read_csv('data\OpenScenRes.csv').drop(columns="Unnamed: 0")

In [4]:
ScenResul_mean =ScenResul.mean()

In [5]:
epsilon = []


## Reference Scenario for optimizer


In [6]:
#load it
with open("data\ScenariosOpenExplo", 'rb') as file2:
    Scenarios = pickle.load(file2)

In [7]:
Scenarios

[Scenario({'discount rate 0': 3.5, 'discount rate 1': 3.5, 'discount rate 2': 3.5, 'A.0_ID flood wave shape': 17, 'A.1_Bmax': 190.0, 'A.1_pfail': 0.5, 'A.1_Brate': 1.5, 'A.2_Bmax': 190.0, 'A.2_pfail': 0.5502055300601616, 'A.2_Brate': 1.5, 'A.3_Bmax': 190.0, 'A.3_pfail': 0.2381, 'A.3_Brate': 1.5, 'A.4_Bmax': 190.0, 'A.4_pfail': 0.5496656662323136, 'A.4_Brate': 1.5, 'A.5_Bmax': 190.0, 'A.5_pfail': 0.1402246519399955, 'A.5_Brate': 1.5}),
 Scenario({'discount rate 0': 3.5, 'discount rate 1': 3.5, 'discount rate 2': 3.5, 'A.0_ID flood wave shape': 17, 'A.1_Bmax': 190.0, 'A.1_pfail': 0.5, 'A.1_Brate': 1.5, 'A.2_Bmax': 190.0, 'A.2_pfail': 0.5, 'A.2_Brate': 1.5, 'A.3_Bmax': 190.0, 'A.3_pfail': 0.14113161725762757, 'A.3_Brate': 1.5, 'A.4_Bmax': 190.0, 'A.4_pfail': 0.5, 'A.4_Brate': 1.5, 'A.5_Bmax': 190.0, 'A.5_pfail': 0.5, 'A.5_Brate': 1.5}),
 Scenario({'discount rate 0': 3.5, 'discount rate 1': 3.5, 'discount rate 2': 3.5, 'A.0_ID flood wave shape': 17, 'A.1_Bmax': 190.0, 'A.1_pfail': 0.5, 'A.

## Optimizer

In [16]:
from ema_workbench.em_framework.optimization import ArchiveLogger, EpsilonProgress

# we need to store our results for each seed
results = []
convergences = []
n = 1

with MultiprocessingEvaluator(dike_model) as evaluator:
    # we run again for 5 seeds
    for i in range(n):
        # we create 2 covergence tracker metrics
        # the archive logger writes the archive to disk for every x nfe
        # the epsilon progress tracks during runtime
        convergence_metrics = [
            ArchiveLogger(
                r"./Archive",
                [l.name for l in dike_model.levers],
                [o.name for o in dike_model.outcomes],
                base_filename=f"{i}.tar.gz",
            ),
            
            EpsilonProgress(),
        ]

        result, convergence = evaluator.optimize(
            nfe=100,
            searchover="levers",
            epsilons=[0.1] * len(dike_model.outcomes),
            convergence=convergence_metrics,
            reference = Scenarios[0]
        )

        results.append(result)
        convergences.append(convergence)

100%|████████████████████████████████████████| 100/100 [00:08<00:00, 12.10it/s]


In [17]:
k = r"./Archive"

## Create complete archive

In [34]:
all_archives = []

for i in range(n):
    archives = ArchiveLogger.load_archives(k+ "\\" + f"{i}.tar.gz")
    print(archives)
    all_archives.append(archives)

{0: Empty DataFrame
Columns: [Unnamed: 0, 0_RfR 0, 0_RfR 1, 0_RfR 2, 1_RfR 0, 1_RfR 1, 1_RfR 2, 2_RfR 0, 2_RfR 1, 2_RfR 2, 3_RfR 0, 3_RfR 1, 3_RfR 2, 4_RfR 0, 4_RfR 1, 4_RfR 2, EWS_DaysToThreat, A.1_DikeIncrease 0, A.1_DikeIncrease 1, A.1_DikeIncrease 2, A.2_DikeIncrease 0, A.2_DikeIncrease 1, A.2_DikeIncrease 2, A.3_DikeIncrease 0, A.3_DikeIncrease 1, A.3_DikeIncrease 2, A.4_DikeIncrease 0, A.4_DikeIncrease 1, A.4_DikeIncrease 2, A.5_DikeIncrease 0, A.5_DikeIncrease 1, A.5_DikeIncrease 2, A.1 Total Costs, A.1_Expected Number of Deaths, A.1_Expected Annual Damage, A.2 Total Costs, A.2_Expected Number of Deaths, A.2_Expected Annual Damage, A.3 Total Costs, A.3_Expected Number of Deaths, A.3_Expected Annual Damage, A.4 Total Costs, A.4_Expected Number of Deaths, A.4_Expected Annual Damage, A.5 Total Costs, A.5_Expected Number of Deaths, A.5_Expected Annual Damage, RfR Total Costs, Expected Evacuation Costs]
Index: []

[0 rows x 49 columns], 100:     Unnamed: 0  0_RfR 0  0_RfR 1  0_RfR 2 

AttributeError: 'list' object has no attribute 'drop'

In [19]:
dike_model_problem =dike_model

In [28]:
results

[    0_RfR 0  0_RfR 1  0_RfR 2  1_RfR 0  1_RfR 1  1_RfR 2  2_RfR 0  2_RfR 1  \
 0         0        1        0        1        0        0        1        0   
 1         1        1        0        1        0        0        1        1   
 2         1        0        1        1        1        0        0        1   
 3         1        0        0        1        0        1        1        0   
 4         1        1        0        0        0        1        1        0   
 ..      ...      ...      ...      ...      ...      ...      ...      ...   
 80        1        1        1        0        0        0        1        1   
 81        0        0        1        0        0        1        1        1   
 82        0        0        1        1        0        0        1        1   
 83        0        1        1        1        1        1        0        1   
 84        0        0        1        1        1        0        0        1   
 
     2_RfR 2  3_RfR 0  ...  A.3_Expected Number of

## Calc metrics

In [29]:
reference_set

,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.3_Expected Number of Deaths,A.3_Expected Annual Damage,A.4 Total Costs,A.4_Expected Number of Deaths,A.4_Expected Annual Damage,A.5 Total Costs,A.5_Expected Number of Deaths,A.5_Expected Annual Damage,RfR Total Costs,Expected Evacuation Costs
0,0,1,0,1,0,0,1,0,0,0,...,0.003374,1.290959e+07,3.955062e+07,0.000000,0.000000e+00,1.134759e+08,0.000000,0.000000e+00,1.222600e+09,1507.003711
1,1,1,0,1,0,0,1,1,0,1,...,0.000000,0.000000e+00,1.521620e+07,0.000036,1.531337e+05,1.865096e+08,0.000000,0.000000e+00,1.068100e+09,6.135670
2,1,0,1,1,1,0,0,1,0,1,...,0.000000,0.000000e+00,4.246564e+07,0.000487,2.124411e+06,1.471984e+08,0.000000,0.000000e+00,8.779000e+08,86.589476
3,1,0,0,1,0,1,1,0,1,0,...,0.000000,0.000000e+00,3.688893e+07,0.000288,4.393024e+05,7.640672e+07,0.000279,2.470332e+05,1.080100e+09,0.000000
4,1,1,0,0,0,1,1,0,0,0,...,0.000000,0.000000e+00,3.799706e+07,0.000000,0.000000e+00,2.209530e+08,0.000000,0.000000e+00,6.601000e+08,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,1,1,1,0,0,0,1,1,1,0,...,0.000000,0.000000e+00,3.181423e+07,0.000000,0.000000e+00,1.795773e+08,0.026135,6.525558e+07,6.020000e+08,2443.659170
81,0,0,1,0,0,1,1,1,0,0,...,0.000000,0.000000e+00,2.845310e+07,0.000000,0.000000e+00,1.275136e+08,0.002418,1.785648e+07,7.411000e+08,1589.867983
82,0,0,1,1,0,0,1,1,0,0,...,0.000000,0.000000e+00,3.958214e+07,0.000003,3.151532e+04,1.560446e+08,0.001906,1.396994e+07,1.132100e+09,1061.656327
83,0,1,1,1,1,1,0,1,0,0,...,0.004981,1.897140e+07,5.677889e+07,0.000000,0.000000e+00,2.550424e+08,0.014983,1.143091e+08,1.621600e+09,11706.433336


In [20]:
from optimization_tuned import (
    HypervolumeMetric,
    GenerationalDistanceMetric,
    EpsilonIndicatorMetric,
    InvertedGenerationalDistanceMetric,
    SpacingMetric,
)
from ema_workbench.em_framework.optimization import to_problem

problem = to_problem(dike_model_problem, searchover="levers")
print(results)

reference_set = epsilon_nondominated(results, [0.1] * (len(dike_model.outcomes)), problem)
print('REFERENCE SET')
print(reference_set)
# hv = HypervolumeMetric(reference_set, problem)
# print(1)
gd = GenerationalDistanceMetric(reference_set, problem, d=1)
print(2)
ei = EpsilonIndicatorMetric(reference_set, problem)
print(3)
ig = InvertedGenerationalDistanceMetric(reference_set, problem, d=1)
print(4)
sm = SpacingMetric(problem)
print(5)


metrics_by_seed = []
for archives in all_archives:
    metrics = []
    for nfe, archive in archives.items():
        print(f'nfe: {nfe}')
        print(f'archive: {archive}')
        if int(nfe) == 0:
            continue
        scores = {
            "generational_distance": gd.calculate(archive),
            "epsilon_indicator": ei.calculate(archive),
            "inverted_gd": ig.calculate(archive),
            "spacing": sm.calculate(archive),
            "nfe": int(nfe),
        }
        metrics.append(scores)
    metrics = pd.DataFrame.from_dict(metrics)

    # sort metrics by number of function evaluations
    metrics.sort_values(by="nfe", inplace=True)
    metrics_by_seed.append(metrics)

[    0_RfR 0  0_RfR 1  0_RfR 2  1_RfR 0  1_RfR 1  1_RfR 2  2_RfR 0  2_RfR 1  \
0         0        1        0        1        0        0        1        0   
1         1        1        0        1        0        0        1        1   
2         1        0        1        1        1        0        0        1   
3         1        0        0        1        0        1        1        0   
4         1        1        0        0        0        1        1        0   
..      ...      ...      ...      ...      ...      ...      ...      ...   
80        1        1        1        0        0        0        1        1   
81        0        0        1        0        0        1        1        1   
82        0        0        1        1        0        0        1        1   
83        0        1        1        1        1        1        0        1   
84        0        0        1        1        1        0        0        1   

    2_RfR 2  3_RfR 0  ...  A.3_Expected Number of Deaths  \
0 

EMAError: The number of columns in the archive (49) does not match the expected number of decision variables and objectives (48).

## Plot

In [ ]:
sns.set_style("white")
fig, axes = plt.subplots(nrows=5, figsize=(8, 12), sharex=True)

ax2, ax3, ax4, ax5, ax6 = axes

for metrics, convergence in zip(metrics_by_seed, convergences):
    # ax1.plot(metrics.nfe, metrics.hypervolume)
    # ax1.set_ylabel("hypervolume")

    ax2.plot(convergence.nfe, convergence.epsilon_progress)
    ax2.set_ylabel("$\epsilon$ progress")

    ax3.plot(metrics.nfe, metrics.generational_distance)
    ax3.set_ylabel("generational distance")

    ax4.plot(metrics.nfe, metrics.epsilon_indicator)
    ax4.set_ylabel("epsilon indicator")

    ax5.plot(metrics.nfe, metrics.inverted_gd)
    ax5.set_ylabel("inverted generational\ndistance")

    ax6.plot(metrics.nfe, metrics.spacing)
    ax6.set_ylabel("spacing")

ax6.set_xlabel("nfe")


sns.despine(fig)

plt.show()

In [ ]:
print(metrics)

In [ ]:

for outcome in dike_model_problem.outcomes:
    print(outcome)
    
#     
# dike_model_problem.outcomes.__delitem__("A.1_Expected Number of Deaths")
# dike_model_problem.outcomes.__delitem__("A.1_Expected Annual Damage")

In [ ]:
# change outcomes so direction is undesirable
minimize = ScalarOutcome.MINIMIZE
maximize = ScalarOutcome.MAXIMIZE

for outcome in dike_model.outcomes:
    if outcome.kind == minimize:
        outcome.kind = maximize
    else:
        outcome.kind = minimize

with MultiprocessingEvaluator(model) as evaluator:
    results1 = evaluator.optimize(
        nfe=1000, searchover="uncertainties", epsilons=[0.1] * len(model.outcomes)
    )

In [ ]:
from ema_workbench.analysis import parcoords

data = results1.loc[:, [o.name for o in model.outcomes]]

# get the minimum and maximum values as present in the dataframe
limits = parcoords.get_limits(data)

# we know that the lowest possible value for all objectives is 0
limits.loc[0, ["utility", "inertia", "reliability", "max_P"]] = 0
# inertia and reliability are defined on unit interval, so their theoretical maximum is 1
limits.loc[1, ["inertia", "reliability"]] = 1

paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(data)
paraxes.invert_axis("max_P")
plt.show()
